In [1]:
from elasticsearch import Elasticsearch, helpers
import json
import pandas as pd
import numpy as np

In [2]:
class es_indexer:
    def __init__(self):
        self.df = pd.read_parquet('bgg_games_info_cleaned.parquet.gzip')
        self.es_client = Elasticsearch(
            "https://localhost:9200",
            basic_auth=("elastic","v48RlBHIb25l_4D0pyz-"),
            ca_certs="~/http_ca.crt"
        )

    def run_indexer(self):
        self.es_client.indices.create(index='bgg', ignore=400)
        self.es_client.indices.delete(index='bgg', ignore=[400, 404])
        self.df['_index'] = 'bgg'
        j = json.loads(self.df.to_json(orient='records'))
        helpers.bulk(self.es_client, j)
es = es_indexer()

In [3]:
es.run_indexer()

C:\Users\NOHP\AppData\Local\Temp\ipykernel_24872\27668650.py:11: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  self.es_client.indices.create(index='bgg', ignore=400)
C:\Users\NOHP\AppData\Local\Temp\ipykernel_24872\27668650.py:12: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  self.es_client.indices.delete(index='bgg', ignore=[400, 404])


In [13]:
import pandas as pd
test_list = []
listing = es.es_client.search(index='bgg', query={
            "constant_score" : { 
                    "filter" : {
                        "terms" : { 
                            "id" : ["224517", "19777"]
                        }
                    }
                }
        })['hits']['hits']
for i in listing:
    test_list.append((
            i['_id'],
            i['_source']['id'],
            i['_source']['name'],
            i['_source']['image']
    ))
listing_df = pd.DataFrame(test_list, columns=['_id', 'bg_id', 'name', 'image'])
listing_df['_id'].to_list()

['LY1j_o8BggURQ-F21Vby', 'Xo1j_o8BggURQ-F21Vf0']

In [14]:
es.es_client.search(index='bgg', query={
            "more_like_this": {
                "fields": ["name", "description", "boardgame_subdomain"],
                "like": [
                    {
                        "_id":'LY1j_o8BggURQ-F21Vby'
                    },
                    {
                        "_id":'fY1j_o8BggURQ-F22FlD'
                    },
                    {
                        "_id":'7I1j_o8BggURQ-F21Vf1'
                    },
                ],
                "min_term_freq": 1,
                "min_doc_freq": 5,
                "max_query_terms": 20
            }
})['hits']['hits'] 


[{'_index': 'bgg',
  '_id': 'QI1j_o8BggURQ-F21Vbz',
  '_score': 40.81641,
  '_ignored': ['description.keyword'],
  '_source': {'id': 28720,
   'bayes_average': 7.96139,
   'year_published': 2007,
   'min_players': 2,
   'max_players': 4,
   'playing_time': 120,
   'min_playtime': 60,
   'max_playtime': 120,
   'age': 14,
   'name': 'Brass: Lancashire',
   'description': "Brass: Lancashire &mdash; first published as Brass &mdash; is an economic strategy game that tells the story of competing cotton entrepreneurs in Lancashire during the industrial revolution. You must develop, build, and establish your industries and network so that you can capitalize demand for iron, coal and cotton. The game is played over two halves: the canal phase and the rail phase. To win the game, score the most victory points (VPs), which are counted at the end of each half. VPs are gained from your canals, rails, and established (flipped) industry tiles. Each round, players take turns according to the turn ord

In [6]:
query_term = 'brass'
es.es_client.search(index='bgg', query={
    "match_all" : {}
        
}, suggest_field='name', suggest_text=query_term, suggest_mode='missing', size=12)['hits']['hits']


[{'_index': 'bgg',
  '_id': 'LY1j_o8BggURQ-F21Vby',
  '_score': 1.0,
  '_ignored': ['description.keyword'],
  '_source': {'id': 224517,
   'bayes_average': 8.4171,
   'year_published': 2018,
   'min_players': 2,
   'max_players': 4,
   'playing_time': 120,
   'min_playtime': 60,
   'max_playtime': 120,
   'age': 14,
   'name': 'Brass: Birmingham',
   'description': "Brass: Birmingham is an economic strategy game sequel to Martin Wallace' 2007 masterpiece, Brass. Brass: Birmingham tells the story of competing entrepreneurs in Birmingham during the industrial revolution, between the years of 1770-1870.<br/><br/>It offers a very different story arc and experience from its predecessor. As in its predecessor, you must develop, build, and establish your industries and network, in an effort to exploit low or high market demands. The game is played over two halves: the canal era (years 1770-1830) and the rail era (years 1830-1870). To win the game, score the most VPs. VPs are counted at the en

In [5]:
query_term = 'brass'
es.es_client.search(index='bgg', query={
    "multi_match" : {
      "query" : query_term,
      "fields" : [ "name^3", "description" ] 
    }
        
}, suggest_field='name', suggest_text=query_term, suggest_mode='missing', size=12)['hits']['hits']


[{'_index': 'bgg',
  '_id': 'LY1j_o8BggURQ-F21Vby',
  '_score': 20.454428,
  '_ignored': ['description.keyword'],
  '_source': {'id': 224517,
   'bayes_average': 8.4171,
   'year_published': 2018,
   'min_players': 2,
   'max_players': 4,
   'playing_time': 120,
   'min_playtime': 60,
   'max_playtime': 120,
   'age': 14,
   'name': 'Brass: Birmingham',
   'description': "Brass: Birmingham is an economic strategy game sequel to Martin Wallace' 2007 masterpiece, Brass. Brass: Birmingham tells the story of competing entrepreneurs in Birmingham during the industrial revolution, between the years of 1770-1870.<br/><br/>It offers a very different story arc and experience from its predecessor. As in its predecessor, you must develop, build, and establish your industries and network, in an effort to exploit low or high market demands. The game is played over two halves: the canal era (years 1770-1830) and the rail era (years 1830-1870). To win the game, score the most VPs. VPs are counted at 